# BedRock user Guide
Bedrock is one of the services in AWS, which provides APIs of many LLMs and VLMs. It allows you to easily integrate the LLMs into your local program and contribute your work with the help of large foundation models. This file provides guidance and tutorial of how to use bedrock service. 

## Choosing the suitable region before going further

Different regions will have the different access of LLMs, please check if the model you want is avaliable or not in the region you chosen before we go further, everything we do next will based on the region you choose. You can check the model support by regions in bedrock in [**here**](https://docs.aws.amazon.com/bedrock/latest/userguide/models-regions.html). You can simply check and change the region on the top right of aws console. 

![jupyter](./pics/regions.png)

## Having a role for the bedrock

Before having accesss of bedrock service, you should have the permissions of bedrock service in your IAM users. **If you do have a IAM user with the access of bedrock, then you can just simply skip this part.** If your don't have IAM user, you should create one for the convenience, since we need to create the access key later (do not recommend to use root user to access bedrock). You can just simply type IAM in the AWS console page to find the IAM service. You can find the user page in the IAM service, which will looks like this.

![jupyter](./pics/IAM_user_page.png)

And following the steps to create a user for the bedrock:

![jupyter](./pics/IAM_user_create.png)

Then create a group and give the bedrock permissions to the group:

![jupyter](./pics/IAM_group_create.png)
![jupyter](./pics/IAM_group_create2.png)

Then add the user to the group:

![jupyter](./pics/IAM_add_user.png)

Create the user:

![jupyter](./pics/IAM_user_create2.png)

Please note that the permissions are added in the group, you will not have the permission if you are not in the group.

Then we create the access key for the user:

![jupyter](./pics/IAM_user_access_key.png)
![jupyter](./pics/IAM_user_access_key2.png)
![jupyter](./pics/IAM_user_access_key3.png)
![jupyter](./pics/IAM_user_access_key4.png)
![jupyter](./pics/IAM_user_access_key5.png)

Please note I highly recommend you to **Download .csv file** and keep it in the safe place, cause you only have one time to copy this access key. If you fail to save or remember it, you may have to recreate new one.

## AWS configuration based on AWS CLI

You need AWS Command Line Interface (AWS CLI) for connect LLMs API on your devices, you can check [here](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html) to learn how to install AWS CLI on your devices. After AWS CLI installed, you still need AWS SDK for Python (Boto3) to use bedrock, please check [here](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) to learn how to install boto3 and config the user on the configuration part of boto3 documentation. One thing you may need to know is the `aws_access_key_id` and `aws_secret_access_key` here will be the access key of the user, should be in the CSV file if you downloaded it. 

## Model access

Before you can use the API of foundation models, you need to have the model access first. You can simply find the model access page in the bedrock service, which is shown below:

![jupyter](./pics/bedrock_model_access.png)

After entring the model access page, you will see the availble models you can use. The image below shows the page of model access, if the access status is `Available to request`, it means you can request the access of this model (on the top of the page to request). If the access status is `Access granted`, it means you already have the access of the API of this specific model, no need to request. And If the access status is `Unavailable`, it means you are not allowed to have access of this model.

![jupyter](./pics/bedrock_avaliable_access.png)

The model is unavailable typically because your regions doesn't support that model, you may change the region to have the access of the model.


## API get

After the model status of target foundation model change to `Access granted`, then you are able to use the API of that model, you can find the model API by open the overview or provider page of the bedrock and click the target model. In here, I'll try to get the API of Sonnet v2 provided by Anthropic.

![jupyter](./pics/get_API.png)


## Using API

Now you can use foundation model API on your own devices. Different model will have different API formate, please check the document of the model you want to use, here, I'll show you an example of how to use Claude 3.5 Sonnet v2. 

In [4]:
import boto3
import json
import base64

Sonnet is a Vison-Language Model, so we need to import based64 to process the images.

In [5]:
bedrock_runtime = boto3.client('bedrock-runtime', region_name='us-east-1')

Please note the `region_name` should be the region you applied for the model access, in here, I use region `'us-ease-1'`.

In [6]:
prompt = "tag this image with only nouns"

In [7]:
with open("image_0002.jpg", "rb") as image_file:
    binary_data = image_file.read()
    base_64_encoded_data = base64.b64encode(binary_data)
    base64_string = base_64_encoded_data.decode('utf-8')

Set the prompt and load the image.

In [8]:
kwargs = {
    "modelId": "anthropic.claude-3-5-sonnet-20240620-v1:0",
    "contentType": "application/json",
    "accept": "application/json",
    "body": json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": base64_string
                        }
                    },
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]
    })
}

Paste the API code here as the value of `kwargs`, but there **two things** need to be noted, first, it's better you convey your inputs as the form of json, second, please remember to change the values of `data` and `text` to your inputs.

In [9]:
response = bedrock_runtime.invoke_model(**kwargs)
body = json.loads(response['body'].read())

Using `invoke_model` to get the response of the model. Loading the results from response.

In [11]:
body

{'id': 'msg_bdrk_014Jrd5zr7pvEDwdcQ2T7URK',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-5-sonnet-20240620',
 'content': [{'type': 'text',
   'text': 'Aircraft\nPlane\nPropeller\nSky\nSmoke'}],
 'stop_reason': 'end_turn',
 'stop_sequence': None,
 'usage': {'input_tokens': 123, 'output_tokens': 16}}

Response form

In [12]:
body['content'][0].get('text')

'Aircraft\nPlane\nPropeller\nSky\nSmoke'

get text response only

## Batch Inference

TBD

## Resource

This tutorial is based on [this video](https://www.youtube.com/watch?v=ab1mbj0acDo), you can see more related bedrock tutorial in that channel.